#### To do:
- get names from file 
- access dbpedia query 
- extract information

#### To do after:
- info extraction from comments 
-- maybe only when infomation is missing

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import rdflib

In [2]:
pop_ents = []
sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-SubNGuess')
sparql.setReturnFormat(JSON)
query = f"""
        SELECT ?s
        WHERE {{
        ?s a <http://example.com/popularEntity> .
            }}
            """
    #print(subject)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
qres = sparql.query().convert() 
for r in qres["results"]["bindings"]:
    pop_ents.append(r["s"]["value"])
len(pop_ents), pop_ents

(200,
 ['http://yago-knowledge.org/resource/15th_Satellite_Awards',
  'http://yago-knowledge.org/resource/25th_United_States_Congress',
  'http://yago-knowledge.org/resource/28th_United_States_Congress',
  'http://yago-knowledge.org/resource/38th_Daytime_Emmy_Awards',
  'http://yago-knowledge.org/resource/55th_Golden_Globe_Awards',
  'http://yago-knowledge.org/resource/A.C._Cesena',
  'http://yago-knowledge.org/resource/A.C._Milan',
  'http://yago-knowledge.org/resource/ACF_Fiorentina',
  'http://yago-knowledge.org/resource/AC_Horsens',
  'http://yago-knowledge.org/resource/AGOVV_Apeldoorn',
  'http://yago-knowledge.org/resource/A_Beautiful_Mind_(film)',
  'http://yago-knowledge.org/resource/About_Schmidt',
  'http://yago-knowledge.org/resource/Adelaide_United_FC',
  'http://yago-knowledge.org/resource/Alexandria,_Louisiana',
  'http://yago-knowledge.org/resource/Ali_Larter',
  'http://yago-knowledge.org/resource/American_Samoa',
  'http://yago-knowledge.org/resource/Ames,_Iowa',
  'ht

In [4]:
s = []
s_source = []
for i in pop_ents:
    topic = i.split('/')[-1]
    split = topic.split('_')
    s.append(' '.join(split))
    
data = {'Entity': pop_ents, 'Label': s}
database=pd.DataFrame(data)
database

,Entity,Label
0,http://yago-knowledge.org/resource/15th_Satell...,15th Satellite Awards
1,http://yago-knowledge.org/resource/25th_United...,25th United States Congress
2,http://yago-knowledge.org/resource/28th_United...,28th United States Congress
3,http://yago-knowledge.org/resource/38th_Daytim...,38th Daytime Emmy Awards
4,http://yago-knowledge.org/resource/55th_Golden...,55th Golden Globe Awards
...,...,...
195,"http://yago-knowledge.org/resource/Yakima,_Was...","Yakima, Washington"
196,http://yago-knowledge.org/resource/Yunjin_Kim,Yunjin Kim
197,http://yago-knowledge.org/resource/Zambia_nati...,Zambia national football team
198,http://yago-knowledge.org/resource/Metro-Goldw...,Metro-Goldwyn-Mayer


In [5]:
len(database)

200

In [6]:
def query(popent):
    fun = False
    while fun == False:
        try:
            sparql = SPARQLWrapper("http://dbpedia.org/sparql")
            sparql.setReturnFormat(JSON)
            sparql.setQuery("""
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                PREFIX dbo: <http://dbpedia.org/ontology/>
                SELECT ?abstract
                WHERE { 
                ?s rdfs:label "%s" @en .
                ?s dbo:abstract ?abstract .
                FILTER (LANG(?abstract) = "en")
                }
                """ %(popent))
            results = sparql.query().convert()
            fun = True
        except:
            fun = False
    return results 

abstracts = []
j = 0
for label in database.Label:
    if j % 1000 == 0:
        print(j)
    pastablen = len(abstracts)
    results = query(label)
    if len(results["results"]["bindings"]) == 0:
        abstracts.append('')
        j+=1
        continue 
    for result in results["results"]["bindings"]:
        abstracts.append(result['abstract']['value'])
        j += 1

0


In [7]:
j, len(database)

(200, 200)

In [8]:
len(database), len(abstracts), len(database.Label)

(200, 200, 200)

In [9]:
database['Abstracts']= abstracts
database

,Entity,Label,Abstracts
0,http://yago-knowledge.org/resource/15th_Satell...,15th Satellite Awards,The 15th Satellite Awards is an award ceremony...
1,http://yago-knowledge.org/resource/25th_United...,25th United States Congress,The 25th United States Congress was a meeting ...
2,http://yago-knowledge.org/resource/28th_United...,28th United States Congress,The 28th United States Congress was a meeting ...
3,http://yago-knowledge.org/resource/38th_Daytim...,38th Daytime Emmy Awards,The 38th Annual Daytime Emmy Awards were held ...
4,http://yago-knowledge.org/resource/55th_Golden...,55th Golden Globe Awards,"The 55th Golden Globe Awards, honoring the bes..."
...,...,...,...
195,"http://yago-knowledge.org/resource/Yakima,_Was...","Yakima, Washington",Yakima (/ˈjækɪmɑː/ or /ˈjækɪmə/) is a city in ...
196,http://yago-knowledge.org/resource/Yunjin_Kim,Yunjin Kim,"Yunjin Kim (Hangul: 김윤진), also known as Kim Yu..."
197,http://yago-knowledge.org/resource/Zambia_nati...,Zambia national football team,The Zambia national football team represents Z...
198,http://yago-knowledge.org/resource/Metro-Goldw...,Metro-Goldwyn-Mayer,"Metro-Goldwyn-Mayer Studios Inc., also known a..."


In [10]:
blanks = 0
for _,label,ab in database.values:
    if len(ab) == 0:
        print(label)
        blanks += 1
'Number of entites with no abstract:',blanks

AGOVV Apeldoorn
Global warming
Grammy Award for Best Dance%2FElectronic Album
PFC Botev Plovdiv
Primetime Emmy Award for Outstanding Lead Actor in a Limited Series or Movie
Primetime Emmy Award for Outstanding Lead Actress in a Limited Series or Movie
Primetime Emmy Award for Outstanding Supporting Actress in a Limited Series or Movie
Warner Home Video


('Number of entites with no abstract:', 8)

In [11]:
database.to_csv('abstracts1.csv')